In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from html_table_parser import parser_functions as parser
import pandas as pd
import math
import time

In [2]:
# 현재 페이지의 표를 데이터 프레임 형으로 반환하는 메소드

def makeTable(mon, year):
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = bs(html,'html.parser')
    table = soup.find(id='mySheet1')#sheet 영역 불러오기
    p=parser.make2d(table)
    df0=pd.DataFrame(p)
    
    # 열 데이터 중복제거
    for i in range(0,33,2):
        df0=df0.drop(i, axis=1)

    df0 = df0.drop(31, axis=1) # 끝에 쓸모 없는거 제거
    df0 = df0.drop(5, axis=1) # 중간 쓸모 없는거 제거


    # 행 데이터 불필요 제거
    for i in range(0,len(df0)):
        if (df0[3][i]==None or df0[3][i]=='' or df0[3][i]=='코드'):
            df0=df0.drop(i)

    df_front=pd.concat([df0[3], df0[7], df0[11], df0[15]], axis=1) 
    df_back=pd.concat([df0[3], df0[7], df0[21], df0[25]], axis=1)

    df_front.columns = ['코드', '품목명', '수출중량', '수입중량']
    df_back.columns = ['코드', '품목명', '수출중량', '수입중량']
    
    #수량 값 float 변환
    df_front['수출중량'] = df_front['수출중량'].str.replace(',','').astype(float)
    df_front['수입중량'] = df_front['수입중량'].str.replace(',','').astype(float)
    
    df_back['수출중량'] = df_back['수출중량'].str.replace(',','').astype(float)
    df_back['수입중량'] = df_back['수입중량'].str.replace(',','').astype(float)
    

    # 물동량 데이터 생성
    #df_front[(year-1)+'_'+mon+'월 '+'물동량'] = df_front['수출중량']+df_front['수입중량']
    #df_back[year+'_'+mon+'월 '+'물동량'] = df_back['수출중량']+df_back['수입중량']
    
    # 인덱스 재설정
    df_front=df_front.rename(columns={'수입중량':str(int(year)-1)+'_'+mon+'월 수입중량'})
    df_front=df_front.rename(columns={'수출중량':str(int(year)-1)+'_'+mon+'월 수출중량'})
    
    df_back=df_back.rename(columns={'수입중량':year+'_'+mon+'월 수입중량'})
    df_back=df_back.rename(columns={'수출중량':year+'_'+mon+'월 수출중량'})
    
    
    df_front = df_front.drop('품목명', axis=1)
    df_front = df_front.set_index('코드')
    
    df_back = df_back.drop('품목명', axis=1)
    df_back = df_back.set_index('코드')
    

    return (df_front, df_back)

In [3]:
df_total=pd.DataFrame()

#크롤링
chromedriver_path = 'C:/Users/종찬/chromedriver_win32/chromedriver.exe'

driver = webdriver.Chrome(chromedriver_path)
driver.implicitly_wait(3)
driver.get('http://stat.kita.net/stat/kts/port/PortItemImpExpList.screen#none')

#국가 전체로 바꾸기
e = driver.find_element_by_xpath('//*[@id="s_ctr_code"]')
e.clear()
e.send_keys("")

#HS 코드 2단위
#hsk = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')
#hsk.send_keys('2')

 #중량 기준 설정
weight = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[3]/select')
weight.send_keys('중량')

#당월 선택
d = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[4]/select')
d.send_keys('당월')

#단위 선택
ton = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[5]/select')
ton.send_keys('t')

#정렬 기준
code = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[6]/select')
code.send_keys('코드')

#오름차순
asc = driver.find_element_by_xpath('//*[@id="s_sort_val_asc"]')
asc.send_keys(' ')
for code in ['50','51','52','53','54','55','56',
             '57','58','59','60','61','62','63',
             '8516','8544','8504','8528', '8501', '8507', '8518', '8529',
            '8541', '8517', '8509', '8536', '8545', '8548', '8503', '8505']:
    
    code_len = str(int(math.log10(int(code)))+1)
    s_code = code

    #시작 코드
    c = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/label[3]/input')
    c.clear()
    c.send_keys(s_code)
    
    hsk = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')
    hsk.send_keys(code_len)
    
    df_final = pd.DataFrame()

    for year in range(10,2,-2):
        str_y = str(2021-year)
        year_xpath = '//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option['+str(year)+']'
        print(str_y+'년 ')
        df_res1 = pd.DataFrame()
        df_res2 = pd.DataFrame()

        for month in range(1,13):
            str_m = str(month)


            print(str_m+'월 진행 중...==== '+s_code+' ====')


            #년월 수정
            year = driver.find_element_by_xpath(year_xpath)
            year.click()

            month = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option['+str_m+']')
            month.click()

            #데이터 수량 조절
            data_num = driver.find_element_by_xpath('//*[@id="listCount"]')
            data_num.send_keys('1')

            time.sleep(1)

            #조회 버튼 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[3]/a/img').click()

            time.sleep(3)

            df_front=makeTable(str_m, str_y)[0]
            df_back=makeTable(str_m, str_y)[1]

            if month==1:
                df_res1 = df_front
                df_res2 = df_back
            else: 
                df_res1 = pd.concat([df_res1,df_front],axis=1)
                df_res2 = pd.concat([df_res2,df_back],axis=1)

        df_result = pd.concat([df_res1, df_res2], axis=1)
        if year==10: df_final = df_result
        else: df_final = pd.concat([df_final, df_result], axis=1)
            
    if code =='50': df_total = df_final.T
    else: df_total=pd.concat([df_total,df_final.T], axis=1)
    print(df_total)

2011년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
6월 진행 중...==== 50 ====
7월 진행 중...==== 50 ====
8월 진행 중...==== 50 ====
9월 진행 중...==== 50 ====
10월 진행 중...==== 50 ====
11월 진행 중...==== 50 ====
12월 진행 중...==== 50 ====
2013년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
6월 진행 중...==== 50 ====
7월 진행 중...==== 50 ====
8월 진행 중...==== 50 ====
9월 진행 중...==== 50 ====
10월 진행 중...==== 50 ====
11월 진행 중...==== 50 ====
12월 진행 중...==== 50 ====
2015년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
6월 진행 중...==== 50 ====
7월 진행 중...==== 50 ====
8월 진행 중...==== 50 ====
9월 진행 중...==== 50 ====
10월 진행 중...==== 50 ====
11월 진행 중...==== 50 ====
12월 진행 중...==== 50 ====
2017년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
6월 진행 중...==== 50 ==

KeyboardInterrupt: 

In [28]:
df_total

코드,50,51,52,53,54,55,56,57,58,59,...,8518,8529,8541,8517,8509,8536,8545,8548,8503,8505
2010_1월 수출중량,6.0,60.0,736.0,9.0,1411.0,1283.0,537.0,462.0,285.0,822.0,...,337.0,1881.0,205.0,527.0,12.0,774.0,8.0,154.0,532.0,49.0
2010_1월 수입중량,62.0,35.0,2786.0,382.0,2022.0,3411.0,1173.0,316.0,274.0,664.0,...,1800.0,631.0,143.0,646.0,162.0,1032.0,923.0,3130.0,349.0,566.0
2010_2월 수출중량,7.0,38.0,367.0,11.0,993.0,1230.0,474.0,390.0,256.0,728.0,...,293.0,1520.0,174.0,341.0,1.0,629.0,28.0,140.0,572.0,23.0
2010_2월 수입중량,50.0,33.0,2493.0,451.0,2059.0,3182.0,1307.0,381.0,189.0,664.0,...,1609.0,663.0,142.0,612.0,106.0,759.0,504.0,2683.0,384.0,467.0
2010_3월 수출중량,10.0,64.0,715.0,4.0,1718.0,1661.0,617.0,611.0,316.0,1095.0,...,348.0,1951.0,213.0,707.0,9.0,681.0,10.0,106.0,531.0,54.0
2010_3월 수입중량,82.0,49.0,3593.0,627.0,2087.0,3979.0,1832.0,353.0,267.0,996.0,...,1931.0,564.0,161.0,675.0,148.0,1079.0,1002.0,3161.0,503.0,542.0
2010_4월 수출중량,9.0,55.0,915.0,5.0,1892.0,1580.0,832.0,383.0,355.0,1470.0,...,428.0,1864.0,219.0,540.0,3.0,881.0,13.0,130.0,648.0,47.0
2010_4월 수입중량,93.0,50.0,3503.0,914.0,2934.0,3934.0,1889.0,307.0,285.0,907.0,...,2169.0,684.0,188.0,686.0,117.0,1200.0,1122.0,3010.0,573.0,729.0
2010_5월 수출중량,6.0,171.0,776.0,3.0,2095.0,1954.0,661.0,298.0,373.0,1376.0,...,398.0,1264.0,187.0,427.0,25.0,812.0,10.0,120.0,571.0,44.0
2010_5월 수입중량,75.0,64.0,3427.0,902.0,2111.0,3866.0,1717.0,291.0,277.0,834.0,...,2323.0,568.0,226.0,586.0,140.0,1167.0,1600.0,1881.0,518.0,745.0


In [30]:
df_total.to_excel('20102017_container.xlsx')

In [4]:
df_total=pd.DataFrame()

#크롤링
chromedriver_path = 'C:/Users/종찬/chromedriver_win32/chromedriver.exe'

driver = webdriver.Chrome(chromedriver_path)
driver.implicitly_wait(3)
driver.get('http://stat.kita.net/stat/kts/port/PortItemImpExpList.screen#none')

#국가 전체로 바꾸기
e = driver.find_element_by_xpath('//*[@id="s_ctr_code"]')
e.clear()
e.send_keys("")

#HS 코드 2단위
#hsk = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')
#hsk.send_keys('2')

 #중량 기준 설정
weight = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[3]/select')
weight.send_keys('중량')

#당월 선택
d = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[4]/select')
d.send_keys('당월')

#단위 선택
ton = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[5]/select')
ton.send_keys('t')

#정렬 기준
code = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[6]/select')
code.send_keys('코드')

#오름차순
asc = driver.find_element_by_xpath('//*[@id="s_sort_val_asc"]')
asc.send_keys(' ')
for code in ['50','51','52','53','54','55','56',
             '57','58','59','60','61','62','63',
             '8516','8544','8504','8528', '8501', '8507', '8518', '8529',
            '8541', '8517', '8509', '8536', '8545', '8548', '8503', '8505']:
    
    code_len = str(int(math.log10(int(code)))+1)
    s_code = code

    #시작 코드
    c = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/label[3]/input')
    c.clear()
    c.send_keys(s_code)
    
    hsk = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[1]/div/select[2]')
    hsk.send_keys(code_len)
    
    df_final = pd.DataFrame()

    for year in range(1,0,-1):
        str_y = str(2021-year)
        year_xpath = '//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[1]/select/option['+str(year)+']'
        print(str_y+'년 ')
        df_res1 = pd.DataFrame()
        df_res2 = pd.DataFrame()

        for month in range(1,6):
            str_m = str(month)


            print(str_m+'월 진행 중...==== '+s_code+' ====')


            #년월 수정
            year = driver.find_element_by_xpath(year_xpath)
            year.click()

            month = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[2]/div[2]/select/option['+str_m+']')
            month.click()

            #데이터 수량 조절
            data_num = driver.find_element_by_xpath('//*[@id="listCount"]')
            data_num.send_keys('1')

            time.sleep(1)

            #조회 버튼 클릭
            driver.find_element_by_xpath('//*[@id="contents"]/div[2]/form/fieldset/div[3]/a/img').click()

            time.sleep(3)

            df_front=makeTable(str_m, str_y)[0]
            df_back=makeTable(str_m, str_y)[1]

            if month==1:
                df_res1 = df_front
                df_res2 = df_back
            else: 
                df_res1 = pd.concat([df_res1,df_front],axis=1)
                df_res2 = pd.concat([df_res2,df_back],axis=1)

        df_result = pd.concat([df_res1, df_res2], axis=1)
        if year==10: df_final = df_result
        else: df_final = pd.concat([df_final, df_result], axis=1)
            
    if code =='50': df_total = df_final.T
    else: df_total=pd.concat([df_total,df_final.T], axis=1)
    print(df_total)

2020년 
1월 진행 중...==== 50 ====
2월 진행 중...==== 50 ====
3월 진행 중...==== 50 ====
4월 진행 중...==== 50 ====
5월 진행 중...==== 50 ====
코드              50
2019_1월 수출중량   0.0
2019_1월 수입중량  30.0
2019_2월 수출중량   1.0
2019_2월 수입중량   8.0
2019_3월 수출중량   2.0
2019_3월 수입중량  29.0
2019_4월 수출중량   1.0
2019_4월 수입중량  23.0
2019_5월 수출중량  14.0
2019_5월 수입중량  15.0
2020_1월 수출중량   0.0
2020_1월 수입중량  20.0
2020_2월 수출중량   2.0
2020_2월 수입중량   1.0
2020_3월 수출중량   1.0
2020_3월 수입중량  27.0
2020_4월 수출중량   0.0
2020_4월 수입중량  11.0
2020_5월 수출중량   0.0
2020_5월 수입중량   5.0
2020년 
1월 진행 중...==== 51 ====
2월 진행 중...==== 51 ====
3월 진행 중...==== 51 ====
4월 진행 중...==== 51 ====
5월 진행 중...==== 51 ====
코드              50     51
2019_1월 수출중량   0.0   22.0
2019_1월 수입중량  30.0   37.0
2019_2월 수출중량   1.0  100.0
2019_2월 수입중량   8.0    5.0
2019_3월 수출중량   2.0   91.0
2019_3월 수입중량  29.0   25.0
2019_4월 수출중량   1.0  158.0
2019_4월 수입중량  23.0   49.0
2019_5월 수출중량  14.0  222.0
2019_5월 수입중량  15.0   49.0
2020_1월 수출중량   0.0   19.0
2020_1월 수입중량  20.0   45.0
2020_2월 수출중량   2.0 

2월 진행 중...==== 58 ====
3월 진행 중...==== 58 ====
4월 진행 중...==== 58 ====
5월 진행 중...==== 58 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   19.0   522.0   141.0  2063.0  1887.0   3

2월 진행 중...==== 62 ====
3월 진행 중...==== 62 ====
4월 진행 중...==== 62 ====
5월 진행 중...==== 62 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   19.0   522.0   141.0  2063.0  1887.0   3

2월 진행 중...==== 8544 ====
3월 진행 중...==== 8544 ====
4월 진행 중...==== 8544 ====
5월 진행 중...==== 8544 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   19.0   522.0   141.0  2063.0  18

2월 진행 중...==== 8501 ====
3월 진행 중...==== 8501 ====
4월 진행 중...==== 8501 ====
5월 진행 중...==== 8501 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   19.0   522.0   141.0  2063.0  18

2월 진행 중...==== 8518 ====
3월 진행 중...==== 8518 ====
4월 진행 중...==== 8518 ====
5월 진행 중...==== 8518 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   19.0   522.0   141.0  2063.0  18

2월 진행 중...==== 8541 ====
3월 진행 중...==== 8541 ====
4월 진행 중...==== 8541 ====
5월 진행 중...==== 8541 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   19.0   522.0   141.0  2063.0  18

2020년 
1월 진행 중...==== 8509 ====
2월 진행 중...==== 8509 ====
3월 진행 중...==== 8509 ====
4월 진행 중...==== 8509 ====
5월 진행 중...==== 8509 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   

2020년 
1월 진행 중...==== 8545 ====
2월 진행 중...==== 8545 ====
3월 진행 중...==== 8545 ====
4월 진행 중...==== 8545 ====
5월 진행 중...==== 8545 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   

2020년 
1월 진행 중...==== 8503 ====
2월 진행 중...==== 8503 ====
3월 진행 중...==== 8503 ====
4월 진행 중...==== 8503 ====
5월 진행 중...==== 8503 ====
코드              50     51      52      53      54      55      56      57  \
2019_1월 수출중량   0.0   22.0   482.0   234.0  2294.0  2777.0   274.0   448.0   
2019_1월 수입중량  30.0   37.0  6928.0  1786.0  5162.0  7134.0  2528.0  1436.0   
2019_2월 수출중량   1.0  100.0   457.0   167.0  1785.0  2325.0   278.0   358.0   
2019_2월 수입중량   8.0    5.0  5424.0  1227.0  3451.0  5084.0  1940.0   905.0   
2019_3월 수출중량   2.0   91.0   555.0   159.0  2059.0  2511.0   347.0   155.0   
2019_3월 수입중량  29.0   25.0  7222.0  1628.0  5135.0  6669.0  2991.0  1293.0   
2019_4월 수출중량   1.0  158.0   754.0   277.0  2860.0  3412.0   514.0   186.0   
2019_4월 수입중량  23.0   49.0  7029.0  2279.0  5421.0  7526.0  3651.0  1559.0   
2019_5월 수출중량  14.0  222.0   813.0   263.0  3495.0  2976.0   809.0   137.0   
2019_5월 수입중량  15.0   49.0  6540.0  2018.0  5090.0  6516.0  3283.0  1714.0   
2020_1월 수출중량   0.0   

In [5]:
df_total.to_excel('2020_container.xlsx')